In [ ]:
import ee
ee.Initialize()
ee.Authenticate()

In [4]:
# location of ship
geometry = ee.Geometry.Point([4.1591602699991, 51.959397608499735])


In [5]:
# viewshed
viewshed = ee.Image('users/fbaart/vaarwegen/viewshed')


In [6]:
# water of the harbor
table = ee.FeatureCollection('users/fbaart/vaarwegen/water')
table.size().getInfo()

226

In [7]:
# rasterize water geometries
water = ee.Image().toByte().paint(table, 1)
# source of the viewshed
source = ee.Image().toByte().paint(geometry, 1, 10)
# mask all other pixels
source = source.updateMask(source)
# compute distance from 
cumulativeCost = water.cumulativeCost(**{
  "source": source,
  "maxDistance": 10 * 1000 # 10 kilometers
})


In [9]:
# export to google drive
task = ee.batch.Export.image.toDrive(
    image=cumulativeCost,  # an ee.Image object.
    region=viewshed.geometry(),
    description='test-export',
    folder='ee-export',
    fileNamePrefix='viewshed-1',
    scale=10,
    crs='EPSG:4326'
)

In [10]:
task.start()